In [14]:
import csv
import datetime
import datetime as dt
import pandas as pd 
import numpy as np
import os
import glob
import yfinance as yf
pd.core.common.is_list_like = pd.api.types.is_list_like
from pandas_datareader.data import DataReader
import pandas_datareader as web
from datetime import date

In [15]:
# get data file names
path =r'/Users/big_u/FINRA DOWNLOAD/CONSOLIDATED FILES'
filenames = glob.glob(path + "/*.txt")

dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename, delimiter = "|"))

# Concatenate all data into one DataFrame
big_frame = pd.concat(dfs, ignore_index=True, sort=True)

In [16]:
#for compatibility with Yahoo Finance
big_frame['Symbol'] = big_frame['Symbol'].str.replace('/','.',regex=False)

In [17]:
big_frame = big_frame.set_index(['Symbol', 'Date'], drop=True)

In [18]:
big_frame=big_frame.sort_values(by=['Symbol', 'Date'])

In [19]:
big_frame = big_frame.drop(columns=['Market', 'ShortExemptVolume'])

In [21]:
big_frame=big_frame.dropna()

In [23]:
big_frame = big_frame.assign(open=np.nan, high=np.nan, low=np.nan, close=np.nan, adj_close=np.nan, volume=np.nan)

In [24]:
big_frame.head()

ShortVolume  TotalVolume  open  high  low  close  adj_close  \
Symbol Date                                                                    
A      20100104     190915.0     649471.0   NaN   NaN  NaN    NaN        NaN   
       20100105     288037.0    1084152.0   NaN   NaN  NaN    NaN        NaN   
       20100106     144831.0     424353.0   NaN   NaN  NaN    NaN        NaN   
       20100107     139776.0     650320.0   NaN   NaN  NaN    NaN        NaN   
       20100108     917926.0    1198368.0   NaN   NaN  NaN    NaN        NaN   

                 volume  
Symbol Date              
A      20100104     NaN  
       20100105     NaN  
       20100106     NaN  
       20100107     NaN  
       20100108     NaN

In [ ]:
for ticker in big_frame.index.get_level_values(0).unique():  # looping through all of the unique tickers 
    if os.path.exists("C:/Users/big_u/FINRA DOWNLOAD/YFDATA/{}.parquet".format(ticker)):
        print("exists")
        continue
    print(ticker) #just a check of where we are at in the loop
    ticker_data = big_frame.copy()[big_frame.index.get_level_values(0) == ticker]  # setting up a temp df with the current ticker 
    ticker_data.reset_index(inplace=True)  # indices become columns in the data 
    ticker_data.drop(columns=["Symbol"], inplace=True)  # dropping the irrelevant index component
    ticker_data['date'] = pd.to_datetime(ticker_data['Date'], format = "%Y%m%d").dt.date
    ticker_data['date'] = ticker_data['date'].astype("datetime64")
    start_date = min(ticker_data["date"])  # finding the minimum date 
    start_date -= dt.timedelta(days=1)  # decrementing by one so min date is included in yfinance pull
    end_date = max(ticker_data["date"])  # finding the max date 
    end_date += dt.timedelta(days=1)   # incrementing by one day so max date is included in yfinance pull 
    for retries in range (0,5):
        try:
            yf_data = yf.download(ticker, start=start_date, end=end_date)  # downloading price data for ticker 
            if yf_data.empty:
                break
        except:
            print ('yfinance JSONDecodeError, retyring: ' + str(retries))
            print ('ticker: ' + ticker + 'start: ' + str(start_date) + ';end: ' + str(end_date))
            time.sleep(5*retries)
    if yf_data.empty:
        continue
    yf_data.reset_index(inplace=True)  # resetting the index, date coming out as a column
    yf_data.columns=["date", "open", "high", "low", "close", "adj_close", "volume"]  # renaming columns 
    yf_data.to_parquet("C:/Users/big_u/FINRA DOWNLOAD/YFDATA/{}.parquet".format(ticker))

exists
A.WD
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- A.WD: No data found, symbol may be delisted
exists
exists
exists
exists
AABA
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- AABA: No data found, symbol may be delisted
exists
exists
AAC.U
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- AAC.U: No data found, symbol may be delisted
AAC.WS
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- AAC.WS: No data found, symbol may be delisted
exists
exists
AACOU
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- AACOU: No data found for this date range, symbol may be delisted
AACOW
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- AACOW: No data found for this date range, symbol may be delisted
exists
exists
AACQW
[*********************100%*